In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

url_1 = "https://www.card-gorilla.com/chart/corp?sub=2&term=monthly&date=2025-05-25" # 신한
url_2 = "https://www.card-gorilla.com/chart/corp?sub=8&term=monthly&date=2025-05-25" # 우리

def get_card_urls_selenium():
    driver = webdriver.Chrome()
    driver.get(url=url_2)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    anchors = soup.select(".num1_card .con_area a[href^='/card/detail/'], .inner .con_area a[href^='/card/detail/']")

    card_urls = {"https://www.card-gorilla.com" + a['href'] for a in anchors}
    driver.quit()
    return list(card_urls)

In [7]:
res = get_card_urls_selenium()
res

AttributeError: 'str' object has no attribute 'capabilities'

In [2]:
def parse_card_detail(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)

    dt_elements = driver.find_elements(By.CSS_SELECTOR, ".lst.bene_area dl dt")
    for dt in dt_elements:
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", dt)
            driver.execute_script("arguments[0].click();", dt)
            time.sleep(0.2)
        except Exception as e:
            print(f"클릭 실패: {e}")

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # 기본 정보
    name = soup.select_one(".data_area .tit .card").get_text(strip=True)
    brand = soup.select_one(".data_area .tit .brand").get_text(strip=True)
    c_brand = soup.select_one(".c_brand span").get_text(strip=True)
    fee_domestic, fee_global = "", ""
    for span in soup.select(".in_out span"):
        text = span.get_text()
        fee = span.select_one("b")
        if not fee:
            continue
        fee_value = fee.get_text(strip=True)
        if "국내" in text:
            fee_domestic = fee_value
        elif "해외" in text:
            fee_global = fee_value
    
    # 혜택 정보
    benefits = []
    for dl in soup.select(".lst.bene_area dl"):
        dt = dl.select_one("dt")
        dd = dl.select_one("dd")
        
        category_tag = dt.select_one("p") if dt else None
        short_desc_tag = dt.select_one("i") if dt else None
        category = category_tag.get_text(strip=True) if category_tag else "기타"
        if category.startswith("유의사항") or category.startswith("꼭 확인"):
            continue

        short_description = short_desc_tag.get_text(strip=True) if short_desc_tag else ""
        detail_description = dd.get_text(strip=True) if dd else ""

        benefits.append({
            "category": category,
            "short_description": short_description,
            "detail_description": detail_description
        })

    return {
        "name": name,                        # 카드명 (예: '쿠팡 와우카드')
        "brand": brand,                      # 카드사 이름 (예: '신한카드')
        "c_brand": c_brand,                  # 국제 브랜드 (예: 'VISA', 'MasterCard')
        "fee_domestic": fee_domestic,        # 국내 전용 연회비
        "fee_global": fee_global,            # 해외 겸용 연회비
        "benefits": benefits,                # 혜택 정보
        "url": url                           # 카드 상세 페이지 URL
    }

In [3]:
import time
import json

card_urls = get_card_urls_selenium()
all_cards = []

for i, url in enumerate(card_urls):
    print(f"Processing card {i + 1}/{len(card_urls)}: Fetching {url}")
    card = parse_card_detail(url)
    if card:
        all_cards.append(card)
    time.sleep(1)

# JSON 파일로 저장
with open("cards_info.json", "w", encoding="utf-8") as f:
    json.dump(all_cards, f, ensure_ascii=False, indent=2)

Processing card 1/10: Fetching https://www.card-gorilla.com/card/detail/2759
Processing card 2/10: Fetching https://www.card-gorilla.com/card/detail/13
Processing card 3/10: Fetching https://www.card-gorilla.com/card/detail/16
Processing card 4/10: Fetching https://www.card-gorilla.com/card/detail/39
Processing card 5/10: Fetching https://www.card-gorilla.com/card/detail/466
Processing card 6/10: Fetching https://www.card-gorilla.com/card/detail/41
Processing card 7/10: Fetching https://www.card-gorilla.com/card/detail/37
Processing card 8/10: Fetching https://www.card-gorilla.com/card/detail/716
Processing card 9/10: Fetching https://www.card-gorilla.com/card/detail/2666
Processing card 10/10: Fetching https://www.card-gorilla.com/card/detail/2690
